# The Traveling Salesman Problem

## This notebook does a few different things to get us to the point where we can visualize the TSP solutions
    ### 1) Calculate distances from the Lat Long corrdinates
    ### 2) Create a Distance Matrix which is needed for the TSP to find a solution
    ### 3) Run the TSP solver
    ### 4) Reindex our Pizza Essentials data by the order suggested from the solver
    ### 5) Save this result as a geodtaframe Shapefile

I will show 3 ways of getting the distance matrix.  They produce similar results.

I will show 2 TSP algorithms. They both produce *very different results*.  

In [1]:
#If needed, install ortools via pip here
#!pip install ortools

In [2]:
import json
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt


## METHOD 1 for Distance Calculation and Matrix 
and
## *METHOD 1 for TSP Algorithm*

### This way uses Python's TSP solver 

https://github.com/dmishin/tsp-solver



In [ ]:
#!pip install tsp_solver2

### Next 2 cells gets the distance matrix using scipy's distance packages directly

In [82]:

from  scipy.spatial.distance import pdist, squareform
df = pd.read_csv("../data/PizzaEssentials_Reduced.csv", usecols=["Latitude", "Longitude"])
df.columns = range(df.shape[1])
print(df)
SCIdist = pdist(df, 'cityblock')
distance_matrix1 = squareform(SCIdist) *1000
print(distance_matrix1)

           0         1
0  -73.99324  40.70258
1  -73.93488  40.79715
2  -73.98130  40.59474
3  -73.99459  40.72301
4  -73.99338  40.66165
5  -73.98697  40.75458
6  -73.93402  40.70492
7  -74.00335  40.73164
8  -73.98382  40.57891
9  -73.99568  40.72157
10 -73.99708  40.71907
11 -73.96154  40.62501
12 -73.99421  40.75565
13 -73.83053  40.70898
14 -73.83279  40.69985
15 -73.88757  40.85555
16 -73.95347  40.71561
17 -74.00029  40.68180
[[  0.     152.9303 119.7803  21.7797  41.0697  58.2703  61.5603  39.1697
  133.0903  21.4297  20.3297 109.2703  54.0397 169.1103 163.1803 258.6403
   52.8003  27.8297]
 [152.9303   0.     248.83   133.85   194.      94.66    93.09   133.98
  267.18   136.38   140.28   198.8    100.83   192.52   199.39   105.71
  100.13   180.76  ]
 [119.7803 248.83     0.     141.56    78.99   165.51   157.46   158.95
   18.35   141.21   140.11    50.03   173.82   265.01   253.62   354.54
  148.7    106.05  ]
 [ 21.7797 133.85   141.56     0.      62.57    39.19    78.66  

#Another way
from scipy.spatial import distance
SCIdist2 = distance.cdist(df,df, 'cityblock') 
distance_matrix2 = (squareform(SCIdist2) *1000).astype(int)

In [83]:
import tsp_solver.greedy_numpy
help(tsp_solver.greedy_numpy)

Help on module tsp_solver.greedy_numpy in tsp_solver:

NAME
    tsp_solver.greedy_numpy

FUNCTIONS
    pairs_by_dist_np(N, distances)
        optimized version of pairs_by_dist, using numpy
    
    solve_tsp(distances, optim_steps=3, pairs_by_dist=<function pairs_by_dist_np at 0x10bc3db00>, endpoints=None)
        Given a distance matrix, finds a solution for the TSP problem.
        Returns list of vertex indices.
        Version that uses Numpy - consumes less memory and works faster.

FILE
    /Users/RoscoeBColtrane/anaconda3/lib/python3.7/site-packages/tsp_solver/greedy_numpy.py




In [84]:
from tsp_solver.greedy_numpy import solve_tsp

optimized_path = solve_tsp(distance_matrix1)  
print(optimized_path)

[8, 2, 11, 4, 17, 0, 10, 9, 3, 7, 12, 5, 16, 6, 1, 15, 13, 14]


In [85]:
df2 = pd.read_csv("../data/PizzaEssentials_Reduced.csv", usecols = ['Longitude', 'Latitude', 'Name','Comment',"URL"])
strs3 = [8, 2, 11, 4, 17, 0, 10, 9, 3, 7, 12, 5, 16, 6, 1, 15, 13, 14]
reindexed = df2.reindex(strs3)

reindexed['ID'] = range(len(reindexed))
reindexed


,Longitude,Latitude,Name,Comment,URL,ID
8,-73.98382,40.57891,Totonno's Pizzeria,Second pizzeria in NY. He was the head chef at...,https://www.google.com/maps/place/Totonno's/da...,0
2,-73.98130,40.59474,L&B Spumoni Gardens,"since 1939, the real deal ice cream of Italy.....",https://www.google.com/maps/place/L%26B+Spumon...,1
11,-73.96154,40.62501,Di Fara Pizza,"OG! this man started it all, and inspired eve...",https://www.google.com/maps/place/Di+Fara+Pizz...,2
4,-73.99338,40.66165,Luigi's Pizza,long family owned joint. still cracking out th...,https://www.google.com/maps/place/Luigi's+Pizz...,3
17,-74.00029,40.68180,Lucali,Place with the fun loving neighborhood focused...,https://www.google.com/maps/place/Lucali/data=...,4
0,-73.99324,40.70258,Grimaldi's Pizzeria,also coal fired. A descendant of Lombardi's pi...,https://www.google.com/maps/place/Grimaldi's+P...,5
10,-73.99708,40.71907,Ferrara Bakery & Cafe,gotta get some pastries. Cannolis,https://www.google.com/maps/place/Ferrara+Bake...,6
9,-73.99568,40.72157,Lombardi's Pizza,first licensed pizzeria in NY. Coal Fired sinc...,https://www.google.com/maps/place/Lombardi's+P...,7
3,-73.99459,40.72301,Prince St. Pizza,NOLITA!!! neighborhood slice joint. reimaginin...,https://www.google.com/maps/place/Prince+St.+P...,8
7,-74.00335,40.73164,John's of Bleecker St.,John's was third pizzeria. John's is prolly th...,https://www.google.com/maps/place/John's+of+Bl...,9


In [86]:
# And also make this a geodataframe and save this as a Shapefile 
import geopandas
import shapely
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(reindexed['Longitude'], reindexed['Latitude'])]

# Convert the count df to geodf
points = geopandas.GeoDataFrame(reindexed, geometry=geometry)
points.head()

,Longitude,Latitude,Name,Comment,URL,ID,geometry
8,-73.98382,40.57891,Totonno's Pizzeria,Second pizzeria in NY. He was the head chef at...,https://www.google.com/maps/place/Totonno's/da...,0,POINT (-73.98382 40.57891)
2,-73.98130,40.59474,L&B Spumoni Gardens,"since 1939, the real deal ice cream of Italy.....",https://www.google.com/maps/place/L%26B+Spumon...,1,POINT (-73.98130 40.59474)
11,-73.96154,40.62501,Di Fara Pizza,"OG! this man started it all, and inspired eve...",https://www.google.com/maps/place/Di+Fara+Pizz...,2,POINT (-73.96154 40.62501)
4,-73.99338,40.66165,Luigi's Pizza,long family owned joint. still cracking out th...,https://www.google.com/maps/place/Luigi's+Pizz...,3,POINT (-73.99338 40.66165)
17,-74.00029,40.68180,Lucali,Place with the fun loving neighborhood focused...,https://www.google.com/maps/place/Lucali/data=...,4,POINT (-74.00029 40.68180)


In [87]:
# MUST SAVE A SHP FOR FUTURE USE AS A GDF
points.to_file("../shapefiles/pizzaTSPgeopandas17-2.shp")
#type(points)

# *METHOD 1 to get the distance matrix*
   ###  USE SCIPY TO FIND MANHATTAN DISTANCE.
   HERE IS A PYTHON SCRIPT THAT CAN BE CALLED TO FIND ALL TYPES OF DISTANCES: HAVERSINCE, EUCLIDEAN, MANHATTAN CITYBLOCK, ETC
   ### https://docs.scipy.org/doc/scipy/reference/spatial.distance.html
   
   Here is cityblock but you can use euclidean too.  Haversine mandates its own import package

In [50]:
from scipy.spatial import distance
df = pd.read_csv('../data/PizzaEssentials_Reduced.csv', usecols=["Latitude", "Longitude"]) #get only 2 columns subset
df

data = distance.cdist(df,df, 'cityblock') * 1000
cityblock = pd.DataFrame(data=data)
cityblock

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.0000,152.9303,119.7803,21.7797,41.0697,58.2703,61.5603,39.1697,133.0903,21.4297,20.3297,109.2703,54.0397,169.1103,163.1803,258.6403,52.8003,27.8297
1,152.9303,0.0000,248.8300,133.8500,194.0000,94.6600,93.0900,133.9800,267.1800,136.3800,140.2800,198.8000,100.8300,192.5200,199.3900,105.7100,100.1300,180.7600
2,119.7803,248.8300,0.0000,141.5600,78.9900,165.5100,157.4600,158.9500,18.3500,141.2100,140.1100,50.0300,173.8200,265.0100,253.6200,354.5400,148.7000,106.0500
3,21.7797,133.8500,141.5600,0.0000,62.5700,39.1900,78.6600,17.3900,154.8700,2.5300,6.4300,131.0500,33.0200,178.0900,184.9600,239.5600,48.5200,46.9100
4,41.0697,194.0000,78.9900,62.5700,0.0000,99.3400,102.6300,79.9600,92.3000,62.2200,61.1200,68.4800,94.8300,210.1800,198.7900,299.7100,93.8700,27.0600
5,58.2703,94.6600,165.5100,39.1900,99.3400,0.0000,102.6100,39.3200,178.8200,41.7200,45.6200,155.0000,8.3100,202.0400,208.9100,200.3700,72.4700,86.1000
6,61.5603,93.0900,157.4600,78.6600,102.6300,102.6100,0.0000,96.0500,175.8100,78.3100,77.2100,107.4300,110.9200,107.5500,106.3000,197.0800,30.1400,89.3900
7,39.1697,133.9800,158.9500,17.3900,79.9600,39.3200,96.0500,0.0000,172.2600,17.7400,18.8400,148.4400,33.1500,195.4800,202.3500,239.6900,65.9100,52.9000
8,133.0903,267.1800,18.3500,154.8700,92.3000,178.8200,175.8100,172.2600,0.0000,154.5200,153.4200,68.3800,187.1300,283.3600,271.9700,372.8900,167.0500,119.3600
9,21.4297,136.3800,141.2100,2.5300,62.2200,41.7200,78.3100,17.7400,154.5200,0.0000,3.9000,130.7000,35.5500,177.7400,184.6100,242.0900,48.1700,44.3800


In [52]:
# make all integers
for col in cityblock.columns.values:
    cityblock[col] = cityblock[col].astype('int64')
cityblock.to_csv("../data/test_matrix_city.csv")
cityblock

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0,152,119,21,41,58,61,39,133,21,20,109,54,169,163,258,52,27
1,152,0,248,133,193,94,93,133,267,136,140,198,100,192,199,105,100,180
2,119,248,0,141,78,165,157,158,18,141,140,50,173,265,253,354,148,106
3,21,133,141,0,62,39,78,17,154,2,6,131,33,178,184,239,48,46
4,41,193,78,62,0,99,102,79,92,62,61,68,94,210,198,299,93,27
5,58,94,165,39,99,0,102,39,178,41,45,154,8,202,208,200,72,86
6,61,93,157,78,102,102,0,96,175,78,77,107,110,107,106,197,30,89
7,39,133,158,17,79,39,96,0,172,17,18,148,33,195,202,239,65,52
8,133,267,18,154,92,178,175,172,0,154,153,68,187,283,271,372,167,119
9,21,136,141,2,62,41,78,17,154,0,3,130,35,177,184,242,48,44


In [53]:
##  FOR HAVERSINE
import math
import numpy
from sklearn.metrics.pairwise import euclidean_distances
from haversine import haversine, Unit
from scipy.spatial import distance

def haversine_manual(coord1, coord2):
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
   
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
   
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
   
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [54]:
%%time
pizza_df = df

# Get the values from the dataframe
ndarray_values = pizza_df.values

# create a matrix
##  using np.zeros creates a matrix and -- as used here -- fills it with zeros the size given by parameters
my_matrix = np.zeros((len(ndarray_values), len(ndarray_values)))

# Loop - these variables contain (index, (longitude, latitude))
# We will calculate the distance between every point to every other point
for outer_idx, outer_var in enumerate(ndarray_values[0:,0:2]):

    # Loop - these variables contain (index, (longitude, latitude))
    for inner_idx, inner_var in enumerate(ndarray_values[0:,0:2]):

        # calculate dist from outer_var to each inner_var and add to matrix
        ##  multiply by 100 to increase size of values to make all integers for TSP calculation
        # FOR HAVERSINE
        my_matrix[outer_idx][inner_idx] = haversine_manual(outer_var, inner_var) 
        #* 1000     


CPU times: user 3.21 ms, sys: 104 µs, total: 3.31 ms
Wall time: 3.74 ms


In [55]:
##### Make sure to change name of output file depending on choes distance calculation ABOVE
results_dataframe = pd.DataFrame(my_matrix) 
results_dataframe.to_csv("../data/test_matrix_Hav.csv")
results_dataframe


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.000000,7111.868239,3565.320150,644.310675,1255.444549,1740.978806,6587.249768,1434.689568,3936.157344,642.509739,661.912233,4254.786121,1631.218202,18098.746596,17846.498374,12660.731514,4441.564479,1010.375117
1,7111.868239,0.000000,8084.729410,7021.102624,7724.651111,5939.640173,2840.490335,7877.009783,8638.266267,7150.110607,7322.330654,6068.005911,6721.111960,11921.477121,11745.760735,5561.498760,3250.727251,8092.328835
2,3565.320150,8084.729410,0.000000,4203.863531,2453.520769,4945.502101,6254.912788,4862.822223,560.889308,4206.994833,4198.681774,2386.355080,5141.462680,17135.537237,16833.112458,13157.944992,4834.053078,3404.908295
3,644.310675,7021.102624,4203.863531,0.000000,1886.678104,1286.922768,6759.868192,1009.631125,4580.165157,129.029128,302.161633,4750.190758,1001.914493,18252.916522,18010.571078,12582.601236,4579.268355,1413.745162
4,1255.444549,7724.651111,2453.520769,1886.678104,0.000000,2938.565619,6734.925030,2415.560401,2752.126907,1855.368051,1808.351698,3715.796388,2884.430511,18171.859749,17900.596455,13194.716848,4738.218327,986.145139
5,1740.978806,5939.640173,4945.502101,1286.922768,2938.565619,0.000000,6083.958447,1952.995879,5401.874268,1401.361449,1565.519208,4881.553637,805.947372,17456.941913,17231.666048,11484.273716,3913.603163,2679.085860
6,6587.249768,2840.490335,6254.912788,6759.868192,6734.925030,6083.958447,0.000000,7754.876792,6758.739175,6877.267844,7027.392195,3925.504002,6873.767294,11511.496830,11260.530776,6946.272449,2188.204894,7405.107931
7,1434.689568,7877.009783,4862.822223,1009.631125,2415.560401,1952.995879,7754.876792,0.000000,5163.410256,907.253090,796.780974,5686.445758,1255.157244,19234.813144,18996.033316,13429.981358,5569.751193,1565.283011
8,3936.157344,8638.266267,560.889308,4580.165157,2752.126907,5401.874268,6758.739175,5163.410256,0.000000,4570.904296,4545.466417,2853.992034,5543.924864,17515.141662,17207.306002,13678.585691,5387.649246,3649.083141
9,642.509739,7150.110607,4206.994833,129.029128,1855.368051,1401.361449,6877.267844,907.253090,4570.904296,0.000000,173.565098,4817.232611,1057.862523,18373.128250,18130.019598,12711.609367,4698.429321,1322.841578


## *Method 2 for Distance Calculation*
#### Run the Distance Calculator either here or via terminal using the .py file
THIS CREATES AN EMPYT MATRIX. THEN POPULATES IT WITH A CHOICE OF DISTANCE CALULATORS FROM THE SCIPY OR HAVERSONE PACKAGES.


In [43]:
#Here is the text of the .py file
import pandas
import numpy as np
import math
import numpy
from sklearn.metrics.pairwise import euclidean_distances
from haversine import haversine, Unit
from scipy.spatial import distance

print("STARTING PROGRAM")

# Define Euclidean Distance formula
#from math import dist
def euclidean(coord1, coord2):   
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    #return np.sqrt(np.sum((coord1-coord2)**2))
    #return euclidean_distances([x1np], [x2np])
    return spatial.distance.euclidean(coord1, coord2)

def haversine_manual(coord1, coord2):
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
   
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
   
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
   
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

if __name__ == '__main__':

    pizza_df = pandas.read_csv('../data/PizzaEssentials_Reduced.csv')
   
    # Get the values from the dataframe
    ndarray_values = pizza_df.values
   
    # create a matrix
    ##  using np.zeros creates a matrix and -- as used here -- fills it with zeros the size given by parameters
    my_matrix = np.zeros((len(ndarray_values), len(ndarray_values)))
   
    # Loop - these variables contain (index, (longitude, latitude))
    # We will calculate the distance between every point to every other point
    for outer_idx, outer_var in enumerate(ndarray_values[0:,0:2]):
         
        # Loop - these variables contain (index, (longitude, latitude))
        for inner_idx, inner_var in enumerate(ndarray_values[0:,0:2]):
           
            # calculate dist from outer_var to each inner_var and add to matrix
            ##  multiply by 100 to increase size of values to make all integers for TSP calculation
            # FOR HAVERSINE
            my_matrix[outer_idx][inner_idx] = haversine(outer_var, inner_var) * 100 
            # FOR EUCLIDEAN
            # my_matrix[outer_idx][inner_idx] = euclidean(outer_var, inner_var, unit='mi') * 1000
            # FOR MANHATTAN
            #my_matrix[outer_idx][inner_idx] = distance.cdist(outer_var, inner_var, 'cityblock') * 1000


    # Save results to file
    ###### Make sure to change name of output file depending on choes distance calculation ABOVE
    results_dataframe = pandas.DataFrame(my_matrix) #, columns={'Longitude','Latitude','Name','Comment','URL'})

    results_dataframe.to_csv("../data/test5.csv")
   
    print("Done")


STARTING PROGRAM
Done


### The result is a distance matix!

In [44]:
df4 = pd.read_csv("../data/test5.csv",header=None, index_col = False)
df4

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,NaN,0.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,5.000000e+00,6.000000e+00,7.000000e+00,8.000000e+00,9.000000e+00,1.000000e+01,1.100000e+01,1.200000e+01,1.300000e+01,1.400000e+01,1.500000e+01,1.600000e+01,1.700000e+01
1,0.0,0.000000e+00,1.113843e+05,2.223941e+05,3.335853e+05,4.447803e+05,5.559758e+05,6.672029e+05,7.783664e+05,8.895613e+05,1.000756e+06,1.111951e+06,1.223151e+06,1.334341e+06,1.445647e+06,1.556831e+06,1.667967e+06,1.779127e+06,1.890317e+06
2,1.0,1.113843e+05,0.000000e+00,1.113147e+05,2.224891e+05,3.336485e+05,4.448179e+05,5.559754e+05,6.672137e+05,7.783844e+05,8.895861e+05,1.000779e+06,1.111955e+06,1.223163e+06,1.334390e+06,1.445580e+06,1.556740e+06,1.667927e+06,1.779136e+06
3,2.0,2.223941e+05,1.113147e+05,0.000000e+00,1.112049e+05,2.223942e+05,3.335858e+05,4.448112e+05,5.559808e+05,6.671705e+05,7.783672e+05,8.895624e+05,1.000758e+06,1.111952e+06,1.223258e+06,1.334441e+06,1.445573e+06,1.556734e+06,1.667927e+06
4,3.0,3.335853e+05,2.224891e+05,1.112049e+05,0.000000e+00,1.111952e+05,2.223918e+05,3.336528e+05,4.447814e+05,5.559767e+05,6.671705e+05,7.783656e+05,8.895681e+05,1.000756e+06,1.112097e+06,1.223275e+06,1.334393e+06,1.445543e+06,1.556731e+06
5,4.0,4.447803e+05,3.336485e+05,2.223942e+05,1.111952e+05,0.000000e+00,1.111974e+05,2.224873e+05,3.335871e+05,4.447816e+05,5.559755e+05,6.671706e+05,7.783735e+05,8.895606e+05,1.000916e+06,1.112090e+06,1.223201e+06,1.334348e+06,1.445536e+06
6,5.0,5.559758e+05,4.448179e+05,3.335858e+05,2.223918e+05,1.111974e+05,0.000000e+00,1.113494e+05,2.223975e+05,3.335854e+05,4.447814e+05,5.559765e+05,6.671764e+05,7.783660e+05,8.897263e+05,1.000898e+06,1.112004e+06,1.223151e+06,1.334342e+06
7,6.0,6.672029e+05,5.559754e+05,4.448112e+05,3.336528e+05,2.224873e+05,1.113494e+05,0.000000e+00,1.114586e+05,2.224581e+05,3.336545e+05,4.448345e+05,5.559836e+05,6.672032e+05,7.784482e+05,8.896296e+05,1.000769e+06,1.111953e+06,1.223167e+06
8,7.0,7.783664e+05,6.672137e+05,5.559808e+05,4.447814e+05,3.335871e+05,2.223975e+05,1.114586e+05,0.000000e+00,1.112159e+05,2.223918e+05,3.335860e+05,4.448040e+05,5.559763e+05,6.674386e+05,7.785886e+05,8.896502e+05,1.000770e+06,1.111951e+06
9,8.0,8.895613e+05,7.783844e+05,6.671705e+05,5.559767e+05,4.447816e+05,3.335854e+05,2.224581e+05,1.112159e+05,0.000000e+00,1.112027e+05,2.223949e+05,3.335942e+05,4.447818e+05,5.562278e+05,6.673739e+05,7.784361e+05,8.895668e+05,1.000757e+06


### If you somehow got an extra index column and header column use this code to remove them
#### SOMETIMES THERE IS A BUNCH OF REDUNDANT DATA CLEANING CREATED BY GETTING THE DISTANCE FORMULA


In [45]:
df4 = df4.drop(df4.columns[[0]], axis=1)
df4 = df4.drop(df4.index[0])
df4

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
1,0.000000e+00,1.113843e+05,2.223941e+05,3.335853e+05,4.447803e+05,5.559758e+05,6.672029e+05,7.783664e+05,8.895613e+05,1.000756e+06,1.111951e+06,1.223151e+06,1.334341e+06,1.445647e+06,1.556831e+06,1.667967e+06,1.779127e+06,1.890317e+06
2,1.113843e+05,0.000000e+00,1.113147e+05,2.224891e+05,3.336485e+05,4.448179e+05,5.559754e+05,6.672137e+05,7.783844e+05,8.895861e+05,1.000779e+06,1.111955e+06,1.223163e+06,1.334390e+06,1.445580e+06,1.556740e+06,1.667927e+06,1.779136e+06
3,2.223941e+05,1.113147e+05,0.000000e+00,1.112049e+05,2.223942e+05,3.335858e+05,4.448112e+05,5.559808e+05,6.671705e+05,7.783672e+05,8.895624e+05,1.000758e+06,1.111952e+06,1.223258e+06,1.334441e+06,1.445573e+06,1.556734e+06,1.667927e+06
4,3.335853e+05,2.224891e+05,1.112049e+05,0.000000e+00,1.111952e+05,2.223918e+05,3.336528e+05,4.447814e+05,5.559767e+05,6.671705e+05,7.783656e+05,8.895681e+05,1.000756e+06,1.112097e+06,1.223275e+06,1.334393e+06,1.445543e+06,1.556731e+06
5,4.447803e+05,3.336485e+05,2.223942e+05,1.111952e+05,0.000000e+00,1.111974e+05,2.224873e+05,3.335871e+05,4.447816e+05,5.559755e+05,6.671706e+05,7.783735e+05,8.895606e+05,1.000916e+06,1.112090e+06,1.223201e+06,1.334348e+06,1.445536e+06
6,5.559758e+05,4.448179e+05,3.335858e+05,2.223918e+05,1.111974e+05,0.000000e+00,1.113494e+05,2.223975e+05,3.335854e+05,4.447814e+05,5.559765e+05,6.671764e+05,7.783660e+05,8.897263e+05,1.000898e+06,1.112004e+06,1.223151e+06,1.334342e+06
7,6.672029e+05,5.559754e+05,4.448112e+05,3.336528e+05,2.224873e+05,1.113494e+05,0.000000e+00,1.114586e+05,2.224581e+05,3.336545e+05,4.448345e+05,5.559836e+05,6.672032e+05,7.784482e+05,8.896296e+05,1.000769e+06,1.111953e+06,1.223167e+06
8,7.783664e+05,6.672137e+05,5.559808e+05,4.447814e+05,3.335871e+05,2.223975e+05,1.114586e+05,0.000000e+00,1.112159e+05,2.223918e+05,3.335860e+05,4.448040e+05,5.559763e+05,6.674386e+05,7.785886e+05,8.896502e+05,1.000770e+06,1.111951e+06
9,8.895613e+05,7.783844e+05,6.671705e+05,5.559767e+05,4.447816e+05,3.335854e+05,2.224581e+05,1.112159e+05,0.000000e+00,1.112027e+05,2.223949e+05,3.335942e+05,4.447818e+05,5.562278e+05,6.673739e+05,7.784361e+05,8.895668e+05,1.000757e+06
10,1.000756e+06,8.895861e+05,7.783672e+05,6.671705e+05,5.559755e+05,4.447814e+05,3.336545e+05,2.223918e+05,1.112027e+05,0.000000e+00,1.111952e+05,2.224216e+05,3.335853e+05,4.451453e+05,5.562584e+05,6.672740e+05,7.783790e+05,8.895608e+05


### Need to change all output matrices to integers,. Use * 100 pr 1000 depending to make all integers, then run on google TSP.

#### Save this as a csv


In [46]:
dfINT = df4.astype(int)
dfINT.to_csv("../data/test_matrix2.csv",encoding='utf-8', index=False)
dfINT

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
1,0,111384,222394,333585,444780,555975,667202,778366,889561,1000755,1111950,1223150,1334340,1445647,1556831,1667966,1779126,1890316
2,111384,0,111314,222489,333648,444817,555975,667213,778384,889586,1000779,1111954,1223163,1334390,1445579,1556739,1667927,1779135
3,222394,111314,0,111204,222394,333585,444811,555980,667170,778367,889562,1000758,1111951,1223258,1334440,1445572,1556734,1667927
4,333585,222489,111204,0,111195,222391,333652,444781,555976,667170,778365,889568,1000755,1112097,1223274,1334392,1445543,1556731
5,444780,333648,222394,111195,0,111197,222487,333587,444781,555975,667170,778373,889560,1000915,1112090,1223200,1334348,1445536
6,555975,444817,333585,222391,111197,0,111349,222397,333585,444781,555976,667176,778365,889726,1000898,1112003,1223151,1334341
7,667202,555975,444811,333652,222487,111349,0,111458,222458,333654,444834,555983,667203,778448,889629,1000768,1111952,1223167
8,778366,667213,555980,444781,333587,222397,111458,0,111215,222391,333585,444804,555976,667438,778588,889650,1000770,1111950
9,889561,778384,667170,555976,444781,333585,222458,111215,0,111202,222394,333594,444781,556227,667373,778436,889566,1000757
10,1000755,889586,778367,667170,555975,444781,333654,222391,111202,0,111195,222421,333585,445145,556258,667273,778379,889560


### DONE WITH THE DISTANCE PORTION  ****************
****************


## *METHOD 1 for TSP Algorithm*
### Using Google's API

NOW THAT WE HAVE THE DISTANCE COLUMN WE CAN TEST OUT THE TRAVELING SALESMAN PROBLEM FOR BEST PATH !

With the TSP solution route order from google's solution __ORtools__ package, we need to apply this order to the dataframe

In [59]:
"""Simple travelling salesman problem between cities."""

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd



def create_data_model():
    """Stores the data for the problem."""
    data = {}
    DM = pd.read_csv('../data/test_matrix_Hav.csv')
    data['distance_matrix'] = DM.values.tolist()  # yapf: disable
    data['num_vehicles'] = 1
    data['depot'] = 5  ## SET THE STARTING POINT
    return data


def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)

if __name__ == '__main__':
    main()

Objective: 17 miles
Route for vehicle 0:
 5 -> 6 -> 7 -> 8 -> 9 -> 10 -> 11 -> 12 -> 13 -> 14 -> 15 -> 16 -> 17 -> 0 -> 1 -> 2 -> 3 -> 4 -> 5



***********************
### Save this output here for future reference in case in reruns it all changes
Tst Matrix 2 = Objective: 3781567 miles
Route for vehicle 0:
 5 -> 4 -> 3 -> 2 -> 0 -> 1 -> 6 -> 13 -> 14 -> 15 -> 16 -> 17 -> 12 -> 11 -> 10 -> 9 -> 8 -> 7 -> 5

TEST Matrix Hav  == Objective: 17 miles
Route for vehicle 0:
 5 -> 6 -> 7 -> 8 -> 9 -> 10 -> 11 -> 12 -> 13 -> 14 -> 15 -> 16 -> 17 -> 0 -> 1 -> 2 -> 3 -> 4 -> 5



Test Matrix City == Objective: 17 miles
Route for vehicle 0:
 5 -> 6 -> 7 -> 8 -> 9 -> 10 -> 11 -> 12 -> 13 -> 14 -> 15 -> 16 -> 17 -> 0 -> 1 -> 2 -> 3 -> 4 -> 5


Test Matrix for Euclidean ==
5 -> 6 -> 7 -> 12 -> 17 -> 16 -> 15 -> 14 -> 13 -> 11 -> 10 -> 9 -> 8 -> 4 -> 3 -> 2 -> 1 -> 0 -> 5
**************************

### NEXT
#### Clean up this output. Transform to list. Then use it to reindex Pizza Dataframe

In [75]:
import re
strs = '  5 -> 4 -> 3 -> 2 -> 0 -> 1 -> 6 -> 13 -> 14 -> 15 -> 16 -> 17 -> 12 -> 11 -> 10 -> 9 -> 8 -> 7 -> 5'
strs = strs.replace('->', ',')
# app.py
def stringToList(string):
    listRes = list(string.split(" , "))
    return listRes

#  THIS OPTION MAKES A LIST
strs1 = stringToList(strs)
print(strs1)

['  5', '4', '3', '2', '0', '1', '6', '13', '14', '15', '16', '17', '12', '11', '10', '9', '8', '7', '5']


In [76]:
# removal of bad_chars using replace() initializing bad_chars_list 
bad_chars = ["'"] 

# using replace() to remove bad_chars  
for i in bad_chars : 
    strs2 = strs.replace(i, '')
print (strs2)

  5 , 4 , 3 , 2 , 0 , 1 , 6 , 13 , 14 , 15 , 16 , 17 , 12 , 11 , 10 , 9 , 8 , 7 , 5


### Then we import the original dataframe and need to reindex it by this new TSP order

In [69]:
pizza_names = pd.read_csv('P../data/izzaEssentials_Reduced.csv', index_col=0)

In [77]:

# Here we reindex the dataframe by the value passed in 
strs3 = [  5 , 4 , 3 , 2 , 0 , 1 , 6 , 13 , 14 , 15 , 16 , 17 , 12 , 11 , 10 , 9 , 8 , 7 , 5]
reindexed = pizza_names.reindex(strs3)
reindexed

,Longitude,Latitude,Name,Comment,URL
5,-73.98697,40.75458,Joe's Pizza,quintessential NY slice. Just lean on an elbow...,https://www.google.com/maps/place/Joe's+Pizza/...
4,-73.99338,40.66165,Luigi's Pizza,long family owned joint. still cracking out th...,https://www.google.com/maps/place/Luigi's+Pizz...
3,-73.99459,40.72301,Prince St. Pizza,NOLITA!!! neighborhood slice joint. reimaginin...,https://www.google.com/maps/place/Prince+St.+P...
2,-73.98130,40.59474,L&B Spumoni Gardens,"since 1939, the real deal ice cream of Italy.....",https://www.google.com/maps/place/L%26B+Spumon...
0,-73.99324,40.70258,Grimaldi's Pizzeria,also coal fired. A descendant of Lombardi's pi...,https://www.google.com/maps/place/Grimaldi's+P...
1,-73.93488,40.79715,Patsy's Pizzeria,an OG joint in East Harlem. prolly the 4 pizze...,https://www.google.com/maps/place/Patsy's+Pizz...
6,-73.93402,40.70492,Roberta's Pizza & Bakery,Neapolitan neighborhood joint,https://www.google.com/maps/place/Roberta's+Pi...
13,-73.83053,40.70898,Dani's House of Pizza,Action Bronson's first joint,https://www.google.com/maps/place/Dani's+House...
14,-73.83279,40.69985,Alfie's Pizza,Action Bronson's second joint,https://www.google.com/maps/place/Alfie's+Pizz...
15,-73.88757,40.85555,Bronx Little Italy,"""Start at prince coffee house on Arthur Avenue...",https://www.google.com/maps/place/Bronx+Little...


In [78]:
##WE NEED TO ADD AN INDEX INT COLUMN TO REORDER THEM
reindexed['ID'] = range(len(reindexed))
reindexed

,Longitude,Latitude,Name,Comment,URL,ID
5,-73.98697,40.75458,Joe's Pizza,quintessential NY slice. Just lean on an elbow...,https://www.google.com/maps/place/Joe's+Pizza/...,0
4,-73.99338,40.66165,Luigi's Pizza,long family owned joint. still cracking out th...,https://www.google.com/maps/place/Luigi's+Pizz...,1
3,-73.99459,40.72301,Prince St. Pizza,NOLITA!!! neighborhood slice joint. reimaginin...,https://www.google.com/maps/place/Prince+St.+P...,2
2,-73.98130,40.59474,L&B Spumoni Gardens,"since 1939, the real deal ice cream of Italy.....",https://www.google.com/maps/place/L%26B+Spumon...,3
0,-73.99324,40.70258,Grimaldi's Pizzeria,also coal fired. A descendant of Lombardi's pi...,https://www.google.com/maps/place/Grimaldi's+P...,4
1,-73.93488,40.79715,Patsy's Pizzeria,an OG joint in East Harlem. prolly the 4 pizze...,https://www.google.com/maps/place/Patsy's+Pizz...,5
6,-73.93402,40.70492,Roberta's Pizza & Bakery,Neapolitan neighborhood joint,https://www.google.com/maps/place/Roberta's+Pi...,6
13,-73.83053,40.70898,Dani's House of Pizza,Action Bronson's first joint,https://www.google.com/maps/place/Dani's+House...,7
14,-73.83279,40.69985,Alfie's Pizza,Action Bronson's second joint,https://www.google.com/maps/place/Alfie's+Pizz...,8
15,-73.88757,40.85555,Bronx Little Italy,"""Start at prince coffee house on Arthur Avenue...",https://www.google.com/maps/place/Bronx+Little...,9


In [79]:
# SAVE THIS NEWLY ORDERED DATFRAME AS the REINDEX OF PIZZA ESSENTIALS DATASET
reindexed.to_csv("../data/Pizza_TSP_Route_4.csv", encoding='utf-8')

### HERE WE MAKE THIS A GEODATAFRAME AND SAVE IT AS A SHAPEFILE

In [80]:
# And also make this a geodataframe and save this as a Shapefile 
import geopandas
import shapely
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(reindexed['Longitude'], reindexed['Latitude'])]

# Convert the count df to geodf
points = geopandas.GeoDataFrame(reindexed, geometry=geometry)
points.head()

,Longitude,Latitude,Name,Comment,URL,ID,geometry
5,-73.98697,40.75458,Joe's Pizza,quintessential NY slice. Just lean on an elbow...,https://www.google.com/maps/place/Joe's+Pizza/...,0,POINT (-73.98697 40.75458)
4,-73.99338,40.66165,Luigi's Pizza,long family owned joint. still cracking out th...,https://www.google.com/maps/place/Luigi's+Pizz...,1,POINT (-73.99338 40.66165)
3,-73.99459,40.72301,Prince St. Pizza,NOLITA!!! neighborhood slice joint. reimaginin...,https://www.google.com/maps/place/Prince+St.+P...,2,POINT (-73.99459 40.72301)
2,-73.98130,40.59474,L&B Spumoni Gardens,"since 1939, the real deal ice cream of Italy.....",https://www.google.com/maps/place/L%26B+Spumon...,3,POINT (-73.98130 40.59474)
0,-73.99324,40.70258,Grimaldi's Pizzeria,also coal fired. A descendant of Lombardi's pi...,https://www.google.com/maps/place/Grimaldi's+P...,4,POINT (-73.99324 40.70258)


In [81]:
# MUST SAVE A SHP FOR FUTURE USE AS A GDF
points.to_file("../shapefiles/pizzaTSPgeopandas17-4.shp")
#type(points)